## 에이전트에 미들웨어 추가하기

에이전트에 미들웨어를 추가하는 방법을 알아봅니다. 미들웨어를 사용하면 로깅, 보안 및 기타 공통적인 관심사를 위해 에이전트 상호 작용을 가로채고 수정할 수 있습니다. 미들웨어에 대한 자세한 내용은 [미들웨어 문서](https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/agent-middleware?pivots=programming-language-python)를 참조하세요.


### 미들웨어 작성하기

간단한 미들웨어를 작성해서 에이전트의 동작을 확장할 수 있습니다. 예를 들어, 에이전트가 응답을 생성하기 전에 로그를 기록하는 미들웨어를 추가할 수 있습니다.

#### Agent 기반 미들웨어

에이전트 기반 미들웨어는 에이전트 실행을 가로채고 수정합니다. 이는 AgentRunContext를 사용하며, AgentRunContext는 다음의 속성을 제공합니다.

- agent: 호출되는 에이전트  
- messages: 대화 내 채팅 메시지 목록  
- is_streaming: 응답이 스트리밍 중인지를 나타내는 Boolean 값  
- metadata: 미들웨어 간에 추가 데이터를 저장하기 위한 딕셔너리  
- result: 에이전트의 응답 (수정 가능)  
- terminate: 추가 처리를 중지하는 플래그  
- kwargs: 에이전트 run 메서드에 전달되는 추가적인 키워드 인수  

In [5]:
from agent_framework import AgentRunContext
from typing import Callable, Awaitable

async def logging_agent_middleware(
    context: AgentRunContext,
    next: Callable[[AgentRunContext], Awaitable[None]],
) -> None:
    """Simple middleware that logs agent execution."""
    print("Agent 작업 시작...")

    # Continue to agent execution
    await next(context)

    print("Agent 작업 종료!")

### Agent를 생성하고, 미들웨어 추가하기

In [ ]:
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    name="GreetingAgent",
    instructions="You are a friendly greeting assistant who responds in korean.",
    middleware=[logging_agent_middleware]   # Add your middleware here
)

async def main():
    result = await agent.run("Hello!")
    print(result.text)

await main()

#### Function 기반 미들웨어

Function 기반의 미들웨어는 에이전트 내의 함수 호출을 가로챕니다. 이는 FunctionInvocationContext를 사용하며, 이 컨텍스트 개체는 다음의 속성을 제공합니다

- function: 호출되는 함수
- arguments: 함수에 대한 검증된 인수
- metadata: 미들웨어 간에 추가 데이터를 저장하기 위한 딕셔너리
- result: 함수의 반환 값 (수정 가능)
- terminate: 추가 처리를 중지하는 플래그
- kwargs: 이 함수를 호출한 채팅 메서드에 전달되는 추가적인 키워드 인수

next 호출은 다음 미들웨어로 넘기거나 혹은 실제 함수를 실행합니다.

In [ ]:
from agent_framework import FunctionInvocationContext
from typing import Callable, Awaitable
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
from agent_framework import agent_middleware, function_middleware, chat_middleware

async def logging_agent_middleware(
    context: AgentRunContext,
    next: Callable[[AgentRunContext], Awaitable[None]],
) -> None:
    """Simple middleware that logs agent execution."""
    print("Agent 작업 시작...")

    # Continue to agent execution
    await next(context)

    print("Agent 작업 종료!")

@function_middleware
async def logging_function_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """Function middleware that logs function execution."""
    # Pre-processing: Log before function execution
    print(f"[Function] Calling {context.function.name}")

    # Continue to next middleware or function execution
    await next(context)

    # Post-processing: Log after function execution
    print(f"[Function] {context.function.name} completed")

agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    name="GreetingAgent",
    instructions="You are a friendly greeting assistant who responds in korean.",
    middleware=[logging_function_middleware, logging_agent_middleware]   # Add your middleware here
)

result = await agent.run("Hello!")
print(result.text)

안녕하세요! 만나서 반가워요. 😊


### 미들웨어와 Function 도구를 사용하는 에이전트 생성

에이전트가 Function 도구를 사용하는 경우, Function 호출도 가로챌 수 있습니다.

In [ ]:
from agent_framework import FunctionInvocationContext

def get_time():
    """Get the current time."""
    from datetime import datetime
    return datetime.now().strftime("%H:%M:%S")

async def logging_function_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """Middleware that logs function calls."""
    print(f"호출한 function : {context.function.name}")

    await next(context)

    print(f"function 결과: {context.result}")

# Add both the function and middleware to your agent
agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    name="TimeAgent",
    instructions="You can tell the current time.",
    tools=[get_time],
    middleware=[logging_function_middleware],
) 
async def main():
    result = await agent.run("What time is it?")
    print(result.text)

await main()

### Run 수준에서 미들웨어 사용
Agent 수준이 아닌 특정 Run 수준에 미들웨어를 추가할 수도 있습니다.

In [ ]:
agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    name="TimeAgent",
    instructions="You can tell the current time."
) 

# Use middleware for this specific run only
result = await agent.run(
    "This is important!",
    middleware=[logging_agent_middleware]
)

async def main():
    result = await agent.run("What time is it?")
    print(result.text)

await main()